MAURY Jocelyn N°2183813

# Analyse des articles scientifiques : Inférence variationnelle et auto-encodeurs variationnels

Pour cette première partie, je vais détailler la substance de 2 des 3 articles scientifiques. Je ne détaillera pas celui sur la création de modèle car je n'ai rien à ajouter de plus qui n'a pas été dis lors du premier rendu. 

# 1. Etat de l'art 

Pour cette première partie, je vais détailler la substance de 2 des 3 articles scientifiques. Je ne détaillera pas celui sur la création de modèle car je n'ai rien à ajouter de plus qui n'a pas été dis lors du premier rendu. 

## a) Variational Inference 

Article : Variational Inference: A Review for Statisticians
Blei, D. M., Kucukelbir, A., & McAuliffe, J. D. (2017). Variational inference: A review for statisticians. Journal of the American statistical Association, 112(518), 859-877.

C'est un ensemble de techniques utilisées pour approximé des densités de probabilités. On l'utilise notamment en Statistiques Bayésiennes pour approximer le postériori. 

Pour estimer notre posterior, plutôt que de passer par du sampling, on va passer par de l'optimisation. 
L'idée est de poser à la base une famille de loi de densités approximatives $Q$. Ensuite, on essaie de trouver le membre de cette famille qui minimise la distance de Kullback-Leibler : 
$$q^{*(z)}= \underset{q(z) \in Q}{\text{arg}} min \  KL \big(q(z)\mid \mid p(z|x)\big)$$

$q$ est la variationnal density. 
On approxime le postérior avec le membre de la famille optimisé $q^*(.)$

La divergence KL se définit comme suit : $$ \operatorname{KL}\left(q(\mathbf{z})\|p(\mathbf{z}|\mathbf{x})\right)=\mathbb{E}[\log q(\mathbf{z})]-\mathbb{E}[\log p(\mathbf{z}|\mathbf{x})]$$

C'est un rapport qui n'est pas symétrique. 

Une des clé de la Variational Inference est de choisir un $Q$ assez flexible pour capturer la densité proche de $p(z|x)$ mais assez simple pour être optimisé efficacement. Donc on continue dans l'approximation pour simplifier les calculs. Attention, on rajoute beaucoup d'ajout d'approximation et d'incertitudes. 

Pour résumer : 
on cherche un membre d'une famille de loi qui approxime au mieux la condition d'intérêt. Une fois cela fait, on s'en sert pour calculer la densité conditionnelle "exacte" (par rapport à notre membre). 

Donc on part de : 
$p(z)$ le prior, choisi par l'utilisateur sur la base de ses croyances 
$p(x)$ le marginale au dénominateur, souvent trop dur à estimer sans être dans un cadre bayésien
$p(x|z)$ la likelihood qui lie prior et les données observées 
$(pz|x)$ le posteriori qu'on cherche à estimer, pour obtenir nos "vraies" variables latentes 

On obtiens la probabilité conditionnelle suivante : 
$$p(z|x) = \frac{p(z,x)}{p(x)}$$
Au numérateur, on a la loi jointe $p(z,x)$ qui est traçable, mais on a au dénominateur $p(x)$ la loi marginale (aussi appellée **évidence** qui n'est pas sûr d'être traçable : $p(x) =\int p(z,x) \, dz$
C'est la qu'on arrive sur une intégrale souvent impossible à calculer (impossible ou trop longue) donc on fait de la Variational Inference, du sampling...

Une première façon de choisir la famille est de se dire qu'on est dans la famille exponentielle qui couvre une large gamme de loi. 

Parmi ces lois figurent la loi gaussienne à partir de laquelle on peut faire des modèle de mixtures gaussiennes. 
Le principe de ce modèle est de se dire qu'il y a plusieurs gaussiennes mélangées à l'œuvre derrière les données. De cette façon, on est plus limité à une seul loi avec un seul jeu de paramètres. 

On note $K$ le nombre de loi gaussienne dans notre modèle 
avec $\mu = {\mu_{1}, \mu_{2}, \dots, \mu_{k}}$. On tire les moyennes de la distribution de notre prior $p(\mu_{K})$. 
On part du principe que ce prior est gaussien : $N(0,\sigma^2)$
On décide de fixer la moyenne à 0 donc de centrer et de laisser la variance en **hyperparamètre**. 
Pour générer une donnée $x_{i}$ : 
- assignation d'un cluster $c_{i}$ à partir d'une distribution catégorielle sur { $1, \dots, K$} 
- Génération d'un donnée à partir de notre loi gaussienne : $N(c_{i}^{T}\mu, 1)$

Le modèle complet ressemble à : 
- $\mu_{k}\sim N(0, \sigma^2),$
- $c_{i} \sim Categorical\left( \frac{1}{k},\dots, \frac{1}{k} \right),$
- $x_{i}|c_{i}, \mu \sim N(c_{i}^{T}\mu, 1)$

La densité jointe des variables latentes et observées est :
$$p(\mu, c, x) = p(\mu) \prod_{i=1}^{n}p(c_{i} p(x_{i}|c_{i}, \mu))$$
Les variables latentes sont $z = \{\mu,c\}$, les $K$ moyennes et la taille de sampling $n$ 

### Evidence Lower Bound (ELBO)
On part de la formule qu'on cherche à optimiser de base : 
$$q^{*(z)}= \underset{q(z) \in Q}{\text{arg}} min \  KL (q(z)\mid \mid p(z|x))$$
On cherche le $q^*(z)$ qui est la loi qui approxime le mieux la loi conditionnelle. Mais cette fonction n'est pas calculable car si on détaille la formule de la Divergence de Kullback-Leibler $\operatorname{KL}$ on a : 
$$ \operatorname{KL}\left(q(\mathbf{z})\|p(\mathbf{z}|\mathbf{x})\right)=\mathbb{E}[\log q(\mathbf{z})]-\mathbb{E}[\log p(\mathbf{z}|\mathbf{x})]$$
SI on cherche à développer $-\mathbb{E}[\log p(\mathbf{z}|\mathbf{x})]$ on a : 
$$-\mathbb{E}[\log p(\mathbf{z}|\mathbf{x})] = -\mathbb{E}\left[ \log\left( \frac{p(z,x)}{p(x)} \right) \right]$$
$$=-\mathbb{E}[\log(p(z,x)) - \log(p(x))]$$

Comme $(\log(p(x)))$ est une constante alors on peut sortir le terme de l'espérance en appliquant le signe $-$ :
$$= -\mathbb{E}[\log(p(z,x))] + \log(p(x))$$
En développant on obtiens :  
$$ \text{KL}\left(q(\mathbf{z})\|p(\mathbf{z}\mid\mathbf{x})\right)=\mathbb{E}\left[\log q(\mathbf{z})\right]-\mathbb{E}\left[\log p(\mathbf{z},\mathbf{x})\right]+\log p(\mathbf{x})$$
Le problème est que $\log(p(x))$ est non traçable, donc on préfère passer par l'ajout d'une constante
$$ \text{KL}\left(q(\mathbf{z})\|p(\mathbf{z}\mid\mathbf{x})\right) - \log(p(x))=\mathbb{E}\left[\log q(\mathbf{z})\right]-\mathbb{E}\left[\log p(\mathbf{z},\mathbf{x})\right]+\log p(\mathbf{x}) - \log(p(x))$$
Ainsi on obtiens : 
$$ \text{KL}\left(q(\mathbf{z})\|p(\mathbf{z}\mid\mathbf{x})\right) - \log(p(x))=\mathbb{E}\left[\log q(\mathbf{z})\right]-\mathbb{E}\left[\log p(\mathbf{z},\mathbf{x})\right]$$
De cette façon, on s'est débarrassé de ce qui était non traçable. Le désavantage de cette technique est qu'on ne calcule plus la $KL$ mais la $KL + \log(p(x))$ avec $\log(p(x))$ qui est une constante. Donc si on arrive à la faire tendre vers 0, on obtiendrai une formule traçable permettant de calculer la $KL$. 
En prenant l'opposé de cette formule, on obtiens l'ELBO : 
$$ELBO(q)= - \big[KL(q(z)| | p(z|x)) \big] + \log p(x)$$
$$ELBO(q) =-\big(\mathbb{E}\big[\log q(z)\big]-\mathbb{E}\big[\log p(z,x)\big] + \log p(x)\big) + \log p(x) $$
$$= - E[\log q(z)] + E[\log p(z,x)] - \log p(x) + \log p(x) $$
$$= E[\log p(z,x)] - E[\log q(z)]$$ 
On appelle cette nouvelle formule l'**ELBO** pour **Evidence Of Lower Bound**. 
 Il reste plus qu'a maximiser le **ELBO** pour minimiser la $KL$ divergence. 
$$ \text{ELBO}(q)=\mathbb{E}[\log p(\mathbf{z},\mathbf{x})]-\mathbb{E}[\log q(\mathbf{z})]$$
On peut réécrire l'**ELBO** comme une somme des espérance de vraisemblances des données et de la $KL$ divergence entre le prior $p(z)$ et de la loi supposée optimale $q(z)$ :
$$\text{ELBO}(q)=\mathbb{E}[\log p(\mathbf{z},\mathbf{x})]-\mathbb{E}[\log q(\mathbf{z})]$$
avec $p(z,x) = p(z)p(x|z)$ on peut écrire : 
$$ELBO(q) = E \big[\log (p(z)p(x|z))  \big] - E\big[\log q(z)  \big]$$
$$= E \big [\log p(z)+\log p(x|x) \big] - E \big[\log q(z) \big]$$
$$ =E \big[\log p(z) \big] + E \big[\log p(x|z) \big] - E\big[\log q(z)\big]$$
Pour rappel : 
$\mathbb{E}\left[\log q(\mathbf{z})\right]-\mathbb{E}\left[\log p(\mathbf{z}|\mathbf{x})\right] = \operatorname{KL}\left(q(\mathbf{z})\right\Vert p(\mathbf{z}|\mathbf{x}))$ 

Donc : 
$$\begin{gathered}
  ELBO(q)= E \big[\log p(z) \big] + E \big[\log p(x|z) \big] - E\big[\log q(z)\big] \newline pour \ rappel \ : \mathbb{E}\left[\log q(\mathbf{z})\right]-\mathbb{E}\left[\log p(\mathbf{z}|\mathbf{x})\right]  = \operatorname{KL}\left(q(\mathbf{z})\right\Vert p(\mathbf{z}|\mathbf{x})) \newline  ELBO(q) =  \mathbb{E}[\log p(\mathbf{x}|\mathbf{z})]-\text{KL}\left(q(\mathbf{z})\|p(\mathbf{z})\right)
\end{gathered} $$
On termine avec : 
$$ELBO(q) = E \big[\log p(x|z) \big] - KL\big(q(z) || p(z)\big)$$
On voit deux choses dans la formules : 
- le 1e terme est l'espérance de la vraisemblance des données. Elle encourage les densités qui placent leurs masses sur les variables latentes expliquant les donnée. 
- le 2e terme est la KL divergence négative entre la densité variationnelle et le prior : elle encourage les densitées proche du prior. 

Ici, on voit bien la balance classique entre la vraisemblance et le prior. Chacun tire vers soit le résultat. 

Une autre propriété importante de l'ELBO est qu'il limite par le bas l'evidence  pour tout $q(z)$ : 
$$ \log p(\mathbf{x})\geq\text{ELBO}(q)$$
On parle de lower-bound de l'évidence. 
Grâce à cette propriété, on se dit que l'ELBO permet de bien approximer la loi marginale $p(x)$

Démonstration : 
$ELBO(q) = - \big[KL \big(q(z)| | p(z|x))\big)\big] + \log p(x)$. La $KL\geq 0$ donc on voit que $\log p(x) = ELBO(q) + \big[KL \big(q(z)| | p(z|x))\big)\big]$
donc $\log p(x) > ELBO(q)$
Si on arrive à minimiser la $KL$ c.a.d $KL = 0$ alors on obtiens $\log p(x) = ELBO(q)$
 Donc on a 2 cas : 
 - SI $KL > 0$ alors 
	 - $\log p(x) > ELBO(q)$
- Si $KL = 0$ alors 
	- $\log p(x) = ELBO(q)$ 

Donc on a bien le résultat suivant : 
$$\log p(\mathbf{x})\geq\text{ELBO}(q)$$
Par cette relation, on se rend compte qu'utiliser l'ELBO est très judicieux car on est capable d'approximer en le maximisant l'**evidence** qui jusque là était intraçable. Cependant, sand doute qu'on arrivera jamais a le maximiser au maximum sinon le KL est à 0 et on a réussi. Plusieurs questions se poent à nous : 
- Comment on fait pour maximiser l'ELBO ? 
	- Qu'est ce qui empêche de toujours l'augmenter ? 
	- Est ce que parfois on bloque dans un maximum local ? 
	- Comment savoir quand on est arrivé au but ou pas ? 
### Parenthèse sur l'algo Algorithm EM

Dans l"équation $\text{ELBO}(q)=\mathbb{E}[\log p(\mathbf{z},\mathbf{x})]-\mathbb{E}[\log q(\mathbf{z})]$ le premier terme correspond à l'espérance de la log-likelihood complète. Ce terme est optimisable via l'Algorithm EM. 
Il respose sur l'idée que $\log p(x) = ELBO(q)$ lorsque que $q(z) = p(z|x)$ (donc $KL =0$). 
Si je comprend bien, pour utiliser l'algorithme EM, on part du principe qu'on a trouvé le prior qui correspond à la vrai loi des variables latentes. 


## Mean-field variationnal family 

Une des façon de résoudre le problème d'optimisation de la Variational Inference dans le cas de MMG est d'utiliser la mean-field variationnal family. Cette façon de faire implique que les variables latentes sont mutuellement indépendantes et que chacun est gouvernée par un facteur distinct de densité variationnelle $q$.
Un membre générique de la mean-field variationnal family est : 
$$q(z) = \prod_{j=1}^{m}q_{j}(z_{j})$$
avec chaque variable latente $z_{i}$ gouvernées par son propre facteur variationnel. 
On voit que les données ($x_{i}$) n'apparaissent pas dans le membre de la mean-field variationnal family. Le lien entre la famille variationnelle et les données se fait à travers le ELBO et le problème de minimisation de la **KL** divergence : $ELBO(q) = E \big[\log p(x|z) \big] - KL\big(q(z) || p(z)\big)$

On n'a pas spécifié la forme paramétrique que peut prendre le facteur variationnel $q_{j}$. En principe, chaque facteur peut prendre n'importe quelle forme paramétrique correspondant à sa variable random. 
Par exemple, une variable latente continue peut prendre en facteur une gaussienne. Une variable latente catégorique peut prendre un facteur catégoriel. Ces choix peuvent déterminer la forme optimale du model de mean-field variationnal family. 

Si on choisit de prendre comme loi la gaussian mixture alors l'approximation des densités est donnée par 
$$q(\mu, c) = \prod_{k=1}^{K}q(\mu_{k};m_{k}, s^{2}_{k})\prod_{i=1}^{n}q(c_{i};\varphi_{i})$$

Pour rappel, $K$ est le nombre de membre de loi dans le modèle. Par exemple $K=10$ veut dire qu'on suppose 10 gaussienne donc aussi 10 cluster possible pour les données. 

Dans le membre on retrouve : 
- $q(\mu_{k};m_{k}, s^{2}_{k})$ 
	- C'est une distribution gaussienne sur le paramètre moyen de la $k^e$ composante de mélange 
		- de moyenne $m_{k}$ 
		- de variance $s_{k}^2$
- $q(c_{i};\varphi_{i})$
	- C'est une distribution sur l'attribution du mélange de la $i^e$ observation
		- Sa probabilité d'assignation est un K-vector $\varphi_{i}$ 

Les composantes de mélanges sont dans notre exemple des gaussiennes avec en paramètres variationnel la moyenne et la variance spécifique au $k^e$ cluster. L'assignation du cluster se faire de manière catégorielle avec en paramètre variationnel la probabilité d'appartenir au cluster pour le $i^e$ point. 

Donc en reprenant les étapes on a : 
$$ \text{ Point de départ : } \quad p(z|x) = \frac{p(z,x)}{p(x)}$$
avec $\log p(x)\geq ELBO(q)$ et  $\log p(z|x) = \log p(z,x) - \log p(x)$ on peut écrire (je remplace $\log p(x)$ par son approximation l'**ELBO**)
$$\log p(z|x) = \log p(z,x) - \log p(x) \geq \log p(z,x) - ELBO(q)$$
Comme $ELBO(q) = E \big[\log p(x|z) \big] - KL\big(q(z) || p(z)\big)$ alors 
$$\log p(z|x) \geq \log p(z,x) - \big(E \big[\log p(x|z) \big] - KL\big(q(z) || p(z)\big)\big)$$
$$ \text{ Application du signe - : } \quad= \log p(z,x) - E \big[\log p(x|z) \big] + KL\big(q(z) || p(z)\big)$$
sachant que  $\operatorname{KL}\left(q(\mathbf{z})\|p(\mathbf{z})\right)=\mathbb{E}[\log q(\mathbf{z})]-\mathbb{E}[\log p(\mathbf{z})]$ alors 
$$\log p(z|x) \geq \log p(z,x) - E \big[\log p(x|z) \big] + \mathbb{E}[\log q(\mathbf{z})]-\mathbb{E}[\log p(\mathbf{z})] $$
il reste  à \définir  $q(z) = \prod_{k=1}^{K}q(\mu_{k};m_{k}, s^{2}_{k})\prod_{i=1}^{n}q(c_{i};\varphi_{i})$ donc 
$$ \text{ Point d'arrivée : } \quad\log p(z|x) \geq \log p(z,x) - E \big[\log p(x|z) \big] + \mathbb{E}[\prod_{k=1}^{K}q(\mu_{k};m_{k}, s^{2}_{k})\prod_{i=1}^{n}q(c_{i};\varphi_{i})]-\mathbb{E}[\log p(\mathbf{z})]$$
On pourrait aussi noter : 
$$\quad\log p(z|x) \geq \log(p(z)*p(x|z))- E \big[\log p(x|z) \big] + \mathbb{E}[\prod_{k=1}^{K}q(\mu_{k};m_{k}, s^{2}_{k})\prod_{i=1}^{n}q(c_{i};\varphi_{i})]-\mathbb{E}[\log p(\mathbf{z})]$$
$$\quad\log p(z|x) \geq \log p(z) + \log p(x|z) - E \big[\log p(x|z) \big] + \mathbb{E}[\prod_{k=1}^{K}q(\mu_{k};m_{k}, s^{2}_{k})\prod_{i=1}^{n}q(c_{i};\varphi_{i})]-\mathbb{E}[\log p(\mathbf{z})]$$

Je me demande s'il y a une relation entre $\log p(x|z)$ et $- E \big[\log p(x|z) \big]$ ainsi qu'entre $\log p(z)$ et $-\mathbb{E}[\log p(\mathbf{z})]$ ?

Un des problèmes de la mean-field variationnal family est qu'elle n'arrive pas a capturer les relations de corrélations entre les composantes

Pour estimer au mieux nos variables latentes selon nos données, on va chercher à maximiser l'ELBO afin qu'il se rapproche au plus de $\log p(x)$.
Pour se faire, on va utiliser le Coordinate Ascent mean-field Variational Inference (CAVI). 

## Coordinate Ascent mean-field Variational Inference (CAVI)

L'algorithme **CAVI** optimise chaque facteur de la densité de la mean-field variationnal family en gardant les autres fixés. Il grimpe l'ELBO jusqu'à un maximum local. 

### Méthode 

On va prendre la $j^e$ variables latente $z_{j}$. La conditionnelle complète de $z_{j}$ est sa densité conditionnelle sachant la densité de toutes les autres variables latentes du modèle ainsi que les observations : $p(z_{j}|z\_{}_{j}, x)$ 
Donc en fixant les autres facteurs variationnel $q_{\ell}(z_{\ell}),\:\ell\neq j$, on obtiens le facteur optimale $q_{j}(z_{j})$ : 
$$ q_{j}^{*}(z_{j}) \propto \exp \left\{\mathbb{E}_{-j}\Big[\log p(z_j\:|\:\mathbf{z}_{-j},\mathbf{x})\Big]\right\}.$$
Donc le facteur optimal pour une variable latente $z_{j}$ donnée est une valeur proportionnelle à l'exponentielle de l'espérance du log de la conditionnelle de cette variable latente sachant tout les autres et les données.
En mean-field variationnal family on suppose que toutes les variables latentes sont indépendantes donc ce résultat est proportionnel à : 
$$ q_j^*(z_j)\propto\exp\left\{\mathbb{E}_{-j}\left[\log p(z_j,\mathbf{z}_{-j},\mathbf{x})\right]\right\}$$
Donc on cherche à optimiser la valeur du paramètre variationnel choisi $q_{j}(z_{j})$ en le fixant et en trouvant les valeurs optimales des autres variables latentes $z\__{j}$.

Ensuite, on s'est dit que ça serait cool de dériver la mise à jour de coordonnées $q_j^*(z_j)\propto\exp\left\{\mathbb{E}_{-j}\left[\log p(z_j,\mathbf{z}_{-j},\mathbf{x})\right]\right\}$. On commence par réécrire l'équation de l'**ELBO** comme une fonction des $j^e$ facteurs variationnels $q_{j}(z_{j})$ et en mettant en constante tout les termes qui n'en dépendent pas : 
$$ELBO(q) = E \big[\log (p(z)p(x|z))  \big] - E\big[\log q(z)  \big]$$
Vers : 
$$ \operatorname{ELBO}(q_j)=\mathbb{E}_j\Big[\mathbb{E}_{-j}\Big[\log p(z_j,\mathbf{z}_{-j},\mathbf{x})\Big]\Big]-\mathbb{E}_j\Big[\log q_j(z_j)\Big]+\mathrm{const}.$$
On prend le deuxième terme $E\big[\log q(z)  \big]$ et on le décompose en deux parties (grâce à l'hypothèse mean-field que les variables sont toutes indépendantes) : 
- Ce qui dépend de $q_{j}(z_{j})$ 
	- $\mathbb{E}_j[\log q_j(z_j)]$
- Ce qui ne dépend pas de $q_{j}(z_{j})$
	- $\mathrm{const}$

On remarque qu'à un ajout de constante prêt, $\operatorname{ELBO}(q_j)=\mathbb{E}_j\Big[\mathbb{E}_{-j}\Big[\log p(z_j,\mathbf{z}_{-j},\mathbf{x})\Big]\Big]-\mathbb{E}_j\Big[\log q_j(z_j)\Big]+\mathrm{const}.$ est égale à $- KL(q_{j}(z_{j})|| q^*_{j}(z_{j}))$ soit : 
$$\operatorname{ELBO}(q_j)=\mathbb{E}_j\Big[\mathbb{E}_{-j}\Big[\log p(z_j,\mathbf{z}_{-j},\mathbf{x})\Big]\Big]-\mathbb{E}_j\Big[\log q_j(z_j)\Big]+\mathrm{const} \underset{\text{à l'ajout d'une constante prêt}}{\approx} - \big(\mathbb{E}\big[q_{j}(z_{j})\big] - \mathbb{E}\big[q^*_{j}(z_{j})\big]\big) $$
en appliquant le signe $-$ on obtiens : 
$$\operatorname{ELBO}(q_j)=\mathbb{E}_j\Big[\mathbb{E}_{-j}\Big[\log p(z_j,\mathbf{z}_{-j},\mathbf{x})\Big]\Big]-\mathbb{E}_j\Big[\log q_j(z_j)\Big]+\mathrm{const} \underset{\text{à l'ajout d'une constante prêt}}{\approx} \mathbb{E}\big[q^{*}_{j}(z_{j})\big] - \mathbb{E}\big[q_{j}(z_{j})\big]$$

L'**ELBO** est généralement une fonction non-convexe donc différente initialisation des valeurs de départ peut mener à des résultats différents à la fin. 
De plus, au lieu de calculer l'ELBO sur l'entièreté du datset, on préfère le faire sur plusieurs petites parties et comparer les différences. Par contre, en faisant le CAVI uniquement sur une partie du dataset, la held-out probability n'est plus assurée d'augmenter à chaque itération.

Durant notre travail, on risque de rencontrer des petits nombres comme on travaille avec des probabilités. Pour éviter les erreurs de calculs, on peut utiliser différentes techniques, come par exemple le log-sum-exp trick.

C'est une technique utilisée pour gérer la manipulation de petits nombre (telle que des probabilité) en évitant les erreurs de calcul informatique du à l'arrondi. 
$$ \log\biggl[\sum_i\exp(x_i)\biggr]=\alpha+\log\biggl[\sum_i\exp(x_i-\alpha)\biggr]$$

La constante $\alpha$ est typiquement initialisée au $max_{i}x_{i}$. De cette façon, on apporte une stabilité numérique aux calculs d'inférence. 

$$ \begin{aligned}\operatorname{ELBO}(\mathbf{m},\mathbf{s}^2,\boldsymbol{\varphi})&=\sum_{k=1}^K\mathbb{E}\big[\log p(\mu_k);m_k,s_k^2\big]\\&+\sum_{i=1}^n(\mathbb{E}\left[\log p(c_i);\varphi_i\right]+\mathbb{E}\big[\log p(x_i\mid c_i,\boldsymbol{\mu});\varphi_i,\mathbf{m},\mathbf{s}^2\big]\big)\\&-\sum_{i=1}^n\mathbb{E}\big[\log q(c_i;\varphi_i)\big]-\sum_{k=1}^K\mathbb{E}\big[\log q(\mu_k;m_k,s_k^2)\big].\end{aligned}$$

## b) Autoencoder variationnel bayésien 

Article : Kingma, D. P., & Welling, M. (2013). Auto-encoding variational bayes. arXiv preprint arXiv:1312.6114.

On a vu en Variational Inference qu'une technique d'approximation commune du postérior est par mean-field variationnal family. Cependant, cette approche nécessite souvent une solution analytique, qui est souvent intraçable. 
On va faire une reparamétrisation afin de construire un estimateur différentiable non biaisé de l'ELBO. Ce sera un estimateur Stochastic Gradient Variational Bayes (SGVB). Il pourra être utilisé pour approximer efficacement le postérior par inférence dans quasiment tout les modèles de variables latentes et/ou paramètres continues. Il sera optimisable avec les techniques classique de Stochastic Gradient Descent. 

L'algorithme proposé pour ce faire est l'Auto-Encoder Variationnal Bayes (AEVB)

## Méthode 

La stratégie employé pour dériver l'**ELBO** fonctionne pour une variété de modèle graphique directionnel avec variables continues. Pour le moment, on va se restreindre au cas commun où l'on a un dataset i.i.d avec des variables latentes par datapoint, et où on voudrais calculer le maximum de vraisemblance ou le maximum a posteriori (MAP) des paramètres globaux par inférence et faire de l'inférence variationnel sur les variables latentes. 

## Cadre 

On se replace dans le cadre de la Variational Inference avec un dataset de $N$ données $x$ d'une certaine variable continue x. On part du principe que la donnée est générée par un processus aléatoire nécessitant une variable aléatoire non observée $z$. 

Le processus est le suivant : 
1. on génère une valeur $z$ d'un certain prior $p_{\theta *}(z)$
2. on génère une valeur x de la distribution conditionnel $p_{\theta *}(x|z)$

On part du principe que le prior $p_{\theta *}(z)$ et la vraisemblance $p_{\theta *}(x|z)$ viennent de famille de distribution paramétrisées $p_{\theta}(z)$ et $p_{\theta}(x|z)$ et que les Probability Density Function (PDF) sont différentiable quasiment partout à la fois pour $\theta$ et pour $z$. 

On se place dans un cadre où : 
- $p(x)$ est intraçable (classique)
- $p_{\theta}(z|x)$ est intraçable (pas d'EM)
- les intégrales pour toute application raisonnables mean-field VB sont intraçables. 
- le dataset a une taille importante (donc pas de sampling) 

## Objectif 

On souhaite résoudre les 3 problèmes suivants : 
1. maximum de vraisemblance ou maximum a posteriori (MAP) efficace pour estimé $\theta$. 
	1. Le paramètre en lui même peut être intéressant, par exemple lorsqu'on analyse un procédé naturel
2. Inférence satisfaisante d'une approximation du postérior pour la variable $z$ sachant les données observées $x$ pour un paramètre choisi $\theta$. 
	1. C'est utile pour coder des données ou faire de la data visualisation 
3. Inférence satisfaisante d'une approximation de la loi marginale de $x$ 
	1. En computer vision, les applications communes sont le denoising, la super-resolution...

On pose $q_{\phi}$ une approximation du posterior intraçable $p_{\theta}(z|x)$. On va utiliser une méthode permettant d'estimer conjointement $\phi$ le paramètre de l'approximation du postérior et le paramètre de génération du modèle $\theta$. 

Ce sont les variables latentes qui permettront d'encoder nos données. Puis a partir de ces variables latentes, on pourra retrouver une bonne partie de l'information de nos données. 

$q_{\phi}(z|x)$ est l'encoder probabiliste parce que sachant une donnée, il produit une distribution sur les valeurs possibles de $z$, a partir duquel aurait été généré $x$. 
$p_{\theta}(x|z)$ est le decoder probabiliste parce que sachant un code $z$ il produit une distribution sur les valeurs possibles de $x$ . 

## L'**ELBO**

La vraisemblance marginale est composée de la somme des vraisemblances marginales sur tout les points : 
$$ \log p_{\boldsymbol{\theta}}(\mathbf{x}^{(1)},\cdots,\mathbf{x}^{(N)})=\sum_{i=1}^{N}\log p_{\boldsymbol{\theta}}(\mathbf{x}^{(i)})$$
qu'on peut réécrire comme : 
$$\log p_{\theta} (x^{(i)}) = D_{KL} (q_{\phi}(z|x^{(i)})\mid\mid p_{\theta}(z|x^{(i)}) + \mathcal{L} (\theta, \phi ; x^{i}) $$
Le $\mathcal{L} (\theta, \phi ; x^{i})$ correspond à l'ELBO. A partir de ma compréhension de l'ancien article, j'avais noté : 
$$ELBO(q)= - \big[KL \big(q(z)| | p(z|x))\big)\big] + \log p(x)$$
Donc dans ce papier l'ELBO est noté par $\mathcal{L} (\theta, \phi ; x^{i})$ 
Alors qu'ici l'ELBO semble être :
$$\mathcal{L} (\theta, \phi ; x^{i}) = - D_{KL} (q_{\phi}(z|x^{(i)})\mid\mid p_{\theta}(z|x^{(i)}) + \log p(x^i) $$
Donc on a une différence entre $q(z)$ et $q(z|x)$. Dans le papier, il est indiqué que ça fonctionne pour $(z|x)$ et que ça pourrait marcher pour $z$. 
Ensuite, on peut réécrire l'ELBO comme :
$$ELBO =  \mathbb{E}_{q_{\phi}(\mathbf{z}|\mathbf{x})}\left[-\log q_{\phi}(\mathbf{z}|\mathbf{x})+\log p_{\theta}(\mathbf{x},\mathbf{z})\right]$$
On peut le développer pour obtenir : 
$$\mathbb{E}[-\log q_{\phi}(\mathbf{z}|\mathbf{x})] + \mathbb{E}[\log p_{\theta}(\mathbf{x},\mathbf{z})]$$
sachant 
$$\mathbb{E}[\log p_{\theta}(\mathbf{x},\mathbf{z})] = \mathbb{E}[\log (p_{\theta}(x|z)p(z))]$$
$$=\mathbb{E}[\log p_{\theta}(x|z) + \log p_{\theta}(z)]$$
$$= \mathbb{E}[\log p_{\theta}(x|z)] + \mathbb{E}[\log p_{\theta}(z)]$$
Si on revient à notre développement initial : 
$$\mathbb{E}[-\log q_{\phi}(\mathbf{z}|\mathbf{x})] + \mathbb{E}[\log p_{\theta}(\mathbf{x},\mathbf{z})]$$
on a maintenant $\mathbb{E}[\log p_{\theta}(\mathbf{x},\mathbf{z})]= \mathbb{E}[\log p_{\theta}(x|z)] + \mathbb{E}[\log p_{\theta}(z)]$ donc 
$$ELBO = \mathbb{E}[-\log q_{\phi}(\mathbf{z}|\mathbf{x})] + \mathbb{E}[\log p_{\theta}(x|z)] + \mathbb{E}[\log p_{\theta}(z)] $$
$$= -\big(\mathbb{E}[\log q_{\phi}(\mathbf{z}|\mathbf{x})] - \mathbb{E}[\log p_{\theta}(z) ] \big) + \mathbb{E}[\log p_{\theta}(x|z)]$$
$$=-D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x})\mid\mid p_{\theta}(z) ) +\mathbb{E}[\log p_{\theta}(x|z)] $$
Ainsi, on voit que d'un côté on observe $-\big(\mathbb{E}[\log q_{\phi}(\mathbf{z}|\mathbf{x})] - \mathbb{E}[\log p_{\theta}(z) ] \big)$ soit la négative divergence entre mon postérior estimée et mon prior. De l'autre on a le terme qui mesure la probabilité que $x$ prenne une valeur sachant une valeur de $z$. Donc plus cette espérance est elevée, meilleure est notre modèle. Cela veut dire que le postérieur est crédible pour nos nouvelles données, sachant que le postérieur est créer à partir du prior et de la vraisemblance de nos données sachant ce prior. Donc on retombe sur le fait qu'on veut maximiser cette espérance ("l'erreur de reconstruction" qui est plutôt la qualité de la reconstruction) et du coup a minimiser la divergence entre notre postérior estimée et notre prior. Sachant que $\log p_{\theta} (x^{(i)}) = D_{KL} (q_{\phi}(z|x^{(i)})\mid\mid p_{\theta}(z|x^{(i)}) + \mathcal{L} (\theta, \phi ; x^{i})$. ON se dit que le premier terme n'est pas calculable, on prend que l'ELBO. 
On veut maximiser l'ELBO donc : 
- maximiser $-D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x})\mid\mid p_{\theta}(z) )$ qui revient à minimiser $D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x})\mid\mid p_{\theta}(z) )$ correspondant à la divergence posterior prior. 
- maximiser $\mathbb{E}[\log p_{\theta}(x|z)]$ correspondant à la qualité de la reconstruction "erreur de reconstruction"

L'expression $D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x})\mid\mid p_{\theta}(z) )$ est souvent intégrable analytiquement. Donc il reste plus qu'à trouver le maximum de $\mathbb{E}[\log p_{\theta}(x|z)]$. On va pouvoir faire par sampling en utilisant un estimateur de gradient stochastique. 

## SGVB estimator and AEVB algorithm 

Sous certaines conditions, pour un certain postérior approximé $q_{\phi}(z|x)$ on peut reparamétrisé la variable aléatoire $\tilde{z}\sim q_{\phi}(z|x)$ en utilisant une transformation différentiable $g_{\phi}(\epsilon, x)$ avec une variable auxiliaire de bruit $\epsilon$ :

$$ \widetilde{\mathbf{z}}=g_{\boldsymbol{\phi}}(\boldsymbol{\epsilon},\mathbf{x})\quad\mathrm{with}\quad\epsilon\sim p(\boldsymbol{\epsilon})$$
Il existe différentes stratégies pour choisir une distribution appropriée $p(\epsilon)$ et une fonction appropriée $g_{\phi}(\epsilon, x)$. 

Maintenant, on peut construire l'estimation de Monte Carlo de l'espérance d'une certaine fonction $f(z)$ w.r.t. $q_{\phi}(z|x)$ comme suit : 
$$ \mathbb{E}_{q_{\boldsymbol{\phi}}(\mathbf{z}|\mathbf{x}^{(i)})}\left[f(\mathbf{z})\right]=\mathbb{E}_{p(\boldsymbol{\epsilon})}\left[f(g_{\boldsymbol{\phi}}(\boldsymbol{\epsilon},\mathbf{x}^{(i)}))\right]\simeq\frac{1}{L}\sum_{l=1}^{L}f(g_{\boldsymbol{\phi}}(\boldsymbol{\epsilon}^{(l)},\mathbf{x}^{(i)}))\quad\mathrm{where}\quad\epsilon^{(l)}\sim p(\boldsymbol{\epsilon})$$

Donc l'espérance sur la loi du postérieur de $f(z)$ revient à l'espérance sur la loi du bruit de $f(g_{\phi}(\epsilon ^{(l)}, x^{(i)}))$. Une bonne estimation de l'espérance étant la moyenne, on finit par : $\frac{1}{L}\sum_{l=1}^{L}f(g_{\boldsymbol{\phi}}(\boldsymbol{\epsilon}^{(l)},\mathbf{x}^{(i)})))$ 

A partir de cette technique, on applique cela sur notre ELBO pour obtenir notre estimateur Stochastic Gradient Variational Bayes (SGVB) $\widetilde{\mathcal{L}}^{A}(\boldsymbol{\theta},\phi;\mathbf{x}^{(i)})\simeq\mathcal{L}(\boldsymbol{\theta},\phi;\mathbf{x}^{(i)})$ : 
Pour rappel : 
$$ELBO =  \mathbb{E}_{q_{\phi}(\mathbf{z}|\mathbf{x})}\left[-\log q_{\phi}(\mathbf{z}|\mathbf{x})+\log p_{\theta}(\mathbf{x},\mathbf{z})\right]$$
En remplaçant $z$ par l'ELBO on obtiens : 
$$ \begin{aligned}&\widetilde{\mathcal{L}}^{A}(\boldsymbol{\theta},\boldsymbol{\phi};\mathbf{x}^{(i)})=\frac{1}{L}\sum_{l=1}^{L}\log p_{\boldsymbol{\theta}}(\mathbf{x}^{(i)},\mathbf{z}^{(i,l)})-\log q_{\boldsymbol{\phi}}(\mathbf{z}^{(i,l)}|\mathbf{x}^{(i)})\\&\mathrm{where}\quad\mathbf{z}^{(i,l)}=g_{\phi}(\boldsymbol{\epsilon}^{(i,l)},\mathbf{x}^{(i)})\quad\mathrm{and}\quad\boldsymbol{\epsilon}^{(l)}\sim p(\boldsymbol{\epsilon})\end{aligned}$$ 
Pour rappel, nous avions decomposer l'ELBO comme suit : 
$$ELBO =-D_{KL}(q_{\phi}(\mathbf{z}|\mathbf{x})\mid\mid p_{\theta}(z) ) +\mathbb{E}[\log p_{\theta}(x|z)] $$
Nous avions aussi vu que la KL divergence peut souvent être intégrée analytiquement. Donc on a que $\mathbb{E}[\log p_{\theta}(x|z)]$ a estimer par sampling. Ainsi, on peut créer un nouvel estimateur de l'ELBO qui a typiquement moins de variance que l'estimateur générique : 
$$ \begin{aligned}&\widetilde{\mathcal{L}}^{B}(\boldsymbol{\theta},\phi;\mathbf{x}^{(i)})=-D_{KL}(q_{\boldsymbol{\phi}}(\mathbf{z}|\mathbf{x}^{(i)})||p_{\boldsymbol{\theta}}(\mathbf{z}))+\frac{1}{L}\sum_{l=1}^{L}(\log p_{\boldsymbol{\theta}}(\mathbf{x}^{(i)}|\mathbf{z}^{(i,l)}))\\&\mathrm{where}\quad\mathbf{z}^{(i,l)}=g_{\boldsymbol{\phi}}(\boldsymbol{\epsilon}^{(i,l)},\mathbf{x}^{(i)})\quad\mathrm{and}\quad\boldsymbol{\epsilon}^{(l)}\sim p(\boldsymbol{\epsilon})\end{aligned}$$


En prenant plusieurs données de notre dataset $X$ avec $N$ données, on peut construire un estimateur du marginal likelihood lower bound de tout notre jeu de données, en se basant sur des mini batchs : 
$$ \mathcal{L}(\boldsymbol{\theta},\boldsymbol{\phi};\mathbf{X})\simeq\widetilde{\mathcal{L}}^M(\boldsymbol{\theta},\boldsymbol{\phi};\mathbf{X}^M)=\frac{N}{M}\sum_{i=1}^M\widetilde{\mathcal{L}}(\boldsymbol{\theta},\boldsymbol{\phi};\mathbf{x}^{(i)})$$
Dans leurs expérimentations les auteurs ont trouvés que le nombre de sample L par données peut être mis à 1 tant que la taille des mini batch est assez grande, par exemple $M = 100$. 

## Reparameterization trick 

On pose $z$ une variable continue aléatoire et $z\sim q_{\phi}(z|x)$ est une certaine distribution conditionnelle. Il est souvent possible d'exprimer $z$ comme une variable déterministe $z = g_{\phi}(\epsilon, x)$ avec $\epsilon$ une variable auxiliaire avec une marginale indépendante $p(\epsilon)$ et $g_{\phi}(.)$ est une certaine fonction évaluée vectoriellement et paramétrisée par $\phi$. Cette reparamétrisation est censée servir pour réécrire l'espérance w.r.t. $q_{\phi}(z|x)$ de façon à ce que l'estimateur de Monte Carlo devienne différentiable w.r.t. $\phi$. 
Grâce à cette reparamétrisatio, nous pourrons faire de la backpropagation par gradient car nous avons rendu la formule différentiable. 